In [1]:
import numpy as np
import scipy.stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import ensemble

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import re
import string
from tqdm.auto import tqdm, trange

sns.set(); 

In [2]:
categorical_columns = ['custom_position', 'region_id', 'operating_schedule_id', 'offer_education_id', 'is_agency', 
                       'is_nonresident', 'is_male', 'driving_license', 'company_id', 'city_id']

def change_columns_type(data):
    # меняем тип некоторых колонок на категориальный
    data['operating_schedule_id'] = data['operating_schedule_id'].astype('object')
    data['is_male'] = data['is_male'].astype('object')
    data['company_id'] = data['company_id'].astype('object')
    data['is_agency'] = data['is_agency'].astype('object')
    data['is_nonresident'] = data['is_nonresident'].astype('object')
    data['offer_education_id'] = data['offer_education_id'].astype('object')
    data['city_id'] = data['city_id'].astype('object')

## Предобработка данных

### Уменьшение вариативности названий вакансий

In [3]:
def remove_punctuation(text):
    string.punctuation += '.'
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

def remove_text_in_parentheses(text):
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    return re.sub("[\(\[].*?[\.\]]", "", text)

In [4]:
def preproccess_custom_position_text(custom_position):
    return remove_multiple_spaces(remove_numbers(remove_punctuation(remove_text_in_parentheses(custom_position)))).lower()

In [5]:
def preproccess_custom_position(data):
    print('Кол-во уникальных названий вакансий до обработки: ', data['custom_position'].nunique())
    data['custom_position'] = data['custom_position'].apply(preproccess_custom_position_text)
    print('Кол-во уникальных названий вакансий до обработки: ', data['custom_position'].nunique())

### Уменьшение кол-ва регионов

In [6]:
def replace_seldom_regions(data):      
    # находим регионы, которые встречаются в данных реже 10 раз
    rvc_less_10 = data['region_id'].value_counts().loc[lambda x : x <= 10]
    regions_less_10 = list(rvc_less_10.keys())
    
    # заменяем регионы, которые встречаются в данных реже 10 раз на регион "Прочее" (0)
    data.loc[ data['region_id'].isin(regions_less_10), 'region_id'] = 0

### Уменьшение кол-ва компаний

In [7]:
def replace_seldom_companies(data):      
    # находим компании, которые встречаются в данных реже 10 раз
    rvc_less_10 = data['company_id'].value_counts().loc[lambda x : x <= 10]
    regions_less_10 = list(rvc_less_10.keys())
    
    # заменяем компании, которые встречаются в данных реже 10 раз на компанию "Прочее" (0)
    data.loc[ data['company_id'].isin(regions_less_10), 'company_id'] = 0

### Уменьшение кол-ва профессий

In [8]:
def delete_seldom_target_prof(data):      
    # находим компании, которые встречаются в данных реже 10 раз
    tpvc_less_5 = data['target_prof'].value_counts().loc[lambda x : x <= 5]
    prof_less_5 = list(tpvc_less_5.keys())
    
    # удаляем строки с профессиями, которые встречаются реже 5 раз
    data.drop( data['target_prof'].isin(prof_less_5).index, inplace=True )    

### Заполнение пропущенных значений

In [9]:
def fill_categorical_features(data):
    # "возраст от" заменяем на моду (наиболее часто встречающееся значение)
    data['age_from'].fillna(int(data['age_from'].mode()), axis=0, inplace=True)
    
    # "возраст до" заменяем на среднее значение
    data['age_to'].fillna(data['age_to'].mean(axis=0), axis=0, inplace=True)
    
    data.loc[ data['offer_experience_year_count'] < 0, 'offer_experience_year_count'] = None
    
    # заменяем требуемый опыт на среднее значение
    data['offer_experience_year_count'].fillna(data['offer_experience_year_count'].mean(axis=0), axis=0, inplace=True)
    
    data['company_id'].fillna(data['company_id'].describe().top, axis=0, inplace=True)
    
    data['driving_license'].fillna(data['driving_license'].describe().top, axis=0, inplace=True)
    
    # -100 - не указано
    data['operating_schedule_id'].fillna(-100, axis=0, inplace=True)
    
    # 0 - любое
    data['offer_education_id'].fillna(0, axis=0, inplace=True)
    
    # для пола заменяем константы True и False на 1 и 0
    data.loc[ data['is_male'] == True, 'is_male'] = 1
    data.loc[ data['is_male'] == False, 'is_male'] = 0
    # для каиегориальной переменной is_male заменяем пропуски на новое значение - 2 (не указан)
    data['is_male'].fillna(2, axis=0, inplace=True) 
    
    # для is_agency заменяем константы True и False на 1 и 0
    data.loc[ data['is_agency'] == True, 'is_agency'] = 1
    data.loc[ data['is_agency'] == False, 'is_agency'] = 0
    # для каиегориальной переменной is_agency заменяем пропуски на самое популярное значение
    data['is_agency'].fillna(data['is_agency'].describe().top, axis=0, inplace=True)
    
    # для is_nonresident заменяем константы True и False на 1 и 0
    data.loc[ data['is_nonresident'] == True, 'is_nonresident'] = 1
    data.loc[ data['is_nonresident'] == False, 'is_nonresident'] = 0

    # для каиегориальной переменной is_nonresident заменяем пропуски на самое популярное значение
    data['is_nonresident'].fillna(data['is_nonresident'].describe().top, axis=0, inplace=True)    

In [10]:
# # заполнение пропущенных значений в разрезе по классам
# def fill_categorical_features_by_target_prof(data):
#     for prof in tqdm(data['target_prof'].unique()):
#         prof_data = data.loc[ data['target_prof'] == prof ]
#         fill_categorical_features(prof_data)

In [11]:
# возвращает бинарные и небинарные категориальные колонки
def get_binary_nonbinary_columns(data):
    change_columns_type(data)
    data_describe = data.describe(include=[object])
    
    binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
    nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
    print('Бинарные категориальные признаки: ', binary_columns)
    print('Небинарные категориальные признаки: ', nonbinary_columns)
    
    return (binary_columns, nonbinary_columns)    

### Нормализация числовых данных

In [12]:
numerical_columns = ['age_from', 'age_to', 'offer_experience_year_count']

In [13]:
def normalize_numerical_columns(data):
    # выделяем датафрейм с числовыми признаками и нормируем значения
    data_numerical = data[numerical_columns]
    scaler = preprocessing.StandardScaler().fit(data_numerical)
    data_numerical = scaler.transform(data_numerical)
    return pd.DataFrame(data_numerical, columns=numerical_columns)

### Векторизация небинарных категориальных признаков

In [14]:
def create_dummy_columns(data, nonbinary_columns):
    # для небинарных признаков создаём фиктивные колонки
    data_nonbinary = pd.get_dummies(data[nonbinary_columns])
    return data_nonbinary

### Объединение всех признаков в один дата-фрейм

In [15]:
def concat_columns(data_numerical, data_binary, data_nonbinary):
    data_all = pd.concat((data_numerical, data_binary, data_nonbinary), axis=1)
    data_all = pd.DataFrame(data_all, dtype=float)
    print('Размер матрицы всех признаков: ', data_all.shape)
    return data_all

In [16]:
def get_target_prof(data):
    # выделяем отдельный вектор-столбец с целевой переменной - target_prof
    y = data['target_prof']
    return y

In [17]:
def get_target_salaries(data):
    # целевые столбцы для предсказания зарплат
    y_sf = data['salary_from']
    y_st = data['salary_to']
    return (y_sf, y_st)

In [18]:
data = pd.read_csv('rabotaru_ru/data/train_public.csv')
#data_test = pd.read_csv('rabotaru_ru/data/test.csv')

In [19]:
%%time
change_columns_type(data)
preproccess_custom_position(data)
#delete_seldom_target_prof(data) # !!!
replace_seldom_regions(data)
replace_seldom_companies(data)
fill_categorical_features(data) 
binary_columns, nonbinary_columns = get_binary_nonbinary_columns(data)
data_numerical = normalize_numerical_columns(data)
data_nonbinary = create_dummy_columns(data, nonbinary_columns)
data_all = concat_columns(data_numerical, data[binary_columns], data_nonbinary)

Кол-во уникальных названий вакансий до обработки:  14345
Кол-во уникальных названий вакансий до обработки:  10641
Бинарные категориальные признаки:  ['is_agency', 'is_nonresident']
Небинарные категориальные признаки:  ['custom_position', 'region_id', 'operating_schedule_id', 'offer_education_id', 'is_male', 'driving_license', 'company_id', 'city_id']
Размер матрицы всех признаков:  (56297, 11506)
Wall time: 4.53 s


In [20]:
#data_all.to_csv('data_preprocessed.csv')

## Балансировка классов

In [21]:
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
#over = SMOTE()
over = RandomOverSampler(random_state=0)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('over', over)] # , ('under', under) , ('model', rf)
pipeline = Pipeline(steps=steps)

In [22]:
X = data_all
y = get_target_prof(data)

In [23]:
%%time
X, y = pipeline.fit_resample(X, y)

MemoryError: Unable to allocate 429. MiB for an array with shape (4882, 11506) and data type float64

## Обучение и валидация модели для проверки её качества

In [20]:
%%time
# Разделение выборок для классификации
X = data_all
y = get_target_prof(data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 11)

Wall time: 5.43 s


In [21]:
%%time
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
rf.fit(X_train, y_train)

Wall time: 5min 11s


RandomForestClassifier(random_state=11)

In [22]:
%%time
y_test_predict = rf.predict(X_test)

Wall time: 2.07 s


In [23]:
acc = accuracy_score(y_test, y_test_predict)
f1 = f1_score(y_test, y_test_predict, average = 'macro')
print(acc, f1)

0.7539964476021315 0.5119519772100021


In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, random_state=11)
clf.fit(X_train, y_train)

## Обучение модели на всех данных и прогнозирование на тестовой выборке

In [ ]:
%%time
X = data_all
y = get_target_prof(data)

from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
rf.fit(X, y)

In [16]:
data = pd.read_csv('rabotaru_ru/data/test.csv')

In [17]:
%%time
change_columns_type(data)
preproccess_custom_position(data)
binary_columns, nonbinary_columns = fill_categorical_features(data)
data_numerical = normalize_numerical_columns(data)
data_nonbinary = create_dummy_columns(data, nonbinary_columns)
data_all = concat_columns(data_numerical, data[binary_columns], data_nonbinary)

Кол-во уникальных названий вакансий до обработки:  5130
Кол-во уникальных названий вакансий до обработки:  4273
Бинарные категориальные признаки:  ['is_agency', 'is_nonresident']
Небинарные категориальные признаки:  ['custom_position', 'region_id', 'operating_schedule_id', 'offer_education_id', 'is_male', 'driving_license', 'company_id', 'city_id']
Размер матрицы всех признаков:  (14074, 8749)
Wall time: 954 ms


In [18]:
X_test = data_all

In [20]:
%%time
# дополняем матрицу тестовой выборки колонками из обучающей для возможности использования обученной модели
missing_cols = set( X.columns ) - set( X_test.columns )
print('Кол-во отсутствующих в тестовой выборке колонок: ', len(list(missing_cols)))

for c in missing_cols:
    X_test[c] = 0
X_test = X_test[X.columns]

Кол-во отсутствующих в тестовой выборке колонок:  17391
Wall time: 1min 31s


In [21]:
%%time
y_test = rf.predict(X_test)

In [22]:
y_test[:10]

array(['водитель', 'упаковщик', 'врач', 'повар', 'заправщик', 'почтальон',
       'рабочий', 'аналитик', 'сварщик', 'столяр'], dtype=object)

In [23]:
# формируем итоговый дата-фрейм с прогнозом
res_columns = ['id','salary_from','salary_to','target_prof']
zero_list =  [0] * len(data['id']) # data['id']

df_res = pd.DataFrame(list(zip(data['id'], zero_list, zero_list, y_test)), 
                      columns=res_columns)
df_res

,id,salary_from,salary_to,target_prof
0,286961,0,0,водитель
1,423834,0,0,упаковщик
2,336464,0,0,врач
3,287634,0,0,повар
4,174883,0,0,заправщик
...,...,...,...,...
14069,290923,0,0,упаковщик
14070,175080,0,0,упаковщик
14071,451820,0,0,менеджер
14072,257668,0,0,помощник


In [24]:
%%time
df_res.to_csv('submit_c.csv', index=False)

Wall time: 20.7 ms
